<font color=red>**Danger zone:**</font> you'll be fine-tuning a model to generate positive, negative or even toxic reviews. We'll be doing this for fun, but this is also the technique for [review bombing](https://en.wikipedia.org/wiki/Review_bomb), bot farms on social media and other less than dignified stuff. It is ultimately your decision how you apply this knowledge, but before you choose, ask yourself: is this why you chose to learn ML?


# LLMs Alignment with Reinforcement Learning from human feedback (RLHF).

_based on the [original notebook](https://github.com/antndlcrx/oxford-llms-workshop/blob/main/materials/seminars/day_3/8_LLMs%20alignment%20with%20RLHF.ipynb) by Ilya Boytsov for the Oxford LLMs workshop_



In this session, you're gonna fine-tune a language model with reinforcement learning to make it generate good (or bad) reviews.

To perform RL-based fine-tuning, we'll use a new (in this course) library called [Transformer Reinforcement Learning (TRL)](https://huggingface.co/docs/trl). TRL implements the main reinforcement learning components of RLHF: reward modeling and fine-tuning with PPO.

![img](https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/images/TRL-readme.png)

In [33]:
# !python -m pip install --upgrade --ignore-installed setuptools accelerate trl==0.7.4 transformers==4.33.1 datasets==2.14.4 peft==0.5.0 tokenizers

### Tutorial: align the model to generate positive movie reviews

To see how TRL works, we'll use it to align GPT2 on IMDB dataset to generate positive (or negative) movie reviews. In fact, __it's your choice whether you want positive or negative reviews.__

But before you choose, let's take a look at the baseline model: a GPT-2 fine-tuned on generating arbitrary movie reviews.

In [34]:
import torch
import transformers
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
main_tokenizer = transformers.AutoTokenizer.from_pretrained("lvwerra/gpt2-imdb")
main_model = transformers.AutoModelForCausalLM.from_pretrained("lvwerra/gpt2-imdb", device_map=device)

In [41]:
inputs = main_tokenizer("The movie", return_tensors='pt').to(device)
generated_ids = main_model.generate(**inputs, max_new_tokens=50, do_sample=True)
print("\nGenerated text:", main_tokenizer.decode(generated_ids.flatten().cpu().numpy().tolist()))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Generated text: The movie was really bad. I found the acting to be average, the storyline to be lame, and the plot to be uninteresting (uncompelling). The acting wasn't great and the film was about as good as Hell. Also this was the worst


If you run this cell a couple of times, you'll see that the model generates both positive, negative and neutral reviews in some proportion. What we're gonna do next is teach the model to generate more positive (or negative) reviews.

Similarly to InstructGPT, we're gonna do that in 2 stages:
- **train a reward model** to assign higher values to positive (or negative) reviews
- fine-tune the language model to **maximize that reward using [proximal policy optimization](https://openai.com/research/openai-baselines-ppo)**



## Stage 1: train a reward model

First, we'll train a BERT-like model as our reward model. We'll generate a synthetic pairwise rankings to emulate human rankings.

__Q:__ why do I need a reward model? Can I just use a pre-trained sentiment classifier? <br> __A:__ Yes, you can - but that only works for movie reviews. But this tutorial will teach you how to do RLHF for any kind objective.


__If you actually want to maximize sentiment (or other "label") instead of human preferences, train reward model as a classifier! (see week5)__


In [46]:
# We'll be fine-tuning a small BERT-like model for now. Please try other models for the main assignment.
reward_model = transformers.AutoModelForSequenceClassification.from_pretrained("distilbert-base-cased", device_map=device)
reward_tokenizer = transformers.AutoTokenizer.from_pretrained("distilbert-base-cased")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


__Note that__ the reward model has a separate tokenizer, different from the main model. They don't need to be the same for RLHF fine-tuning.

In [48]:
# To train a reward model, you need a dataset (or generator) of positive-negative pairs.
# Each training sample should be a dict with 4 keys:
#  - input_ids_chosen, attention_mask_chosen = tokenizer("A sentence that human labeler likes more")
#  - input_ids_rejected, attention_mask_rejected = tokenizer("A sentence that human labeler likes less")

import torch
import datasets

class IMDBPairwiseDataset(torch.utils.data.Dataset):
    """ A dataset of all possible pairs of chosen and texts in TRT reward training format """
    def __init__(self, imdb, tokenizer, accepted_label: int):
        super().__init__()
        self.tokenizer = tokenizer
        self.chosen_texts = [row['text'] for row in imdb if row['label'] == accepted_label]
        self.rejected_texts = [row['text'] for row in imdb if row['label'] != accepted_label]
        assert self.chosen_texts, f"no texts with label {accepted_label}"
        print(f"Found {len(self.chosen_texts)} chosen and {len(self.rejected_texts)} rejected texts, {len(self)} pairs")

    def __len__(self):
        return len(self.chosen_texts) * len(self.rejected_texts)  # all pairs

    def __getitem__(self, index: int):
        chosen = self.tokenizer(self.chosen_texts[index // len(self.chosen_texts)], truncation=True)
        rejected = self.tokenizer(self.rejected_texts[index % len(self.chosen_texts)], truncation=True)
        return dict(input_ids_chosen=chosen['input_ids'], attention_mask_chosen=chosen['attention_mask'],
                    input_ids_rejected=rejected['input_ids'], attention_mask_rejected=rejected['attention_mask'])

In [49]:
TARGET_LABEL = 0   # and make sure it works by reviewing the sample printed below
imdb = datasets.load_dataset("imdb", split='train')
reward_data = IMDBPairwiseDataset(imdb, reward_tokenizer, accepted_label=TARGET_LABEL)

sample = reward_data[31337]
print('CHOSEN:', reward_tokenizer.decode(sample['input_ids_chosen']))
print('REJECTED:', reward_tokenizer.decode(sample['input_ids_rejected']))

Found 12500 chosen and 12500 rejected texts, 156250000 pairs
CHOSEN: [CLS] If only to avoid making this type of film in the future. This film is interesting as an experiment but tells no cogent story. < br / > < br / > One might feel virtuous for sitting thru it because it touches on so many IMPORTANT issues but it does so without any discernable motive. The viewer comes away with no new perspectives ( unless one comes up with one while one's mind wanders, as it will invariably do during this pointless film ). < br / > < br / > One might better spend one's time staring out a window at a tree growing. < br / > < br / > [SEP]
REJECTED: [CLS] This movie has some things that are pretty amazing. First, it is supposed to be based on a true story. That, in itself, is amazing that multiple tornadoes would hit the same town at night in the fall - in Nebraska. I wonder if the real town's name was close to " Blainsworth " ( which is the town's name in the movie ). There is an Ainsworth, Nebraska,

We'll be using `trl.RewardTrainer` - a special case of `transformers.Trainer` that you used in the past. `RewardTrainer` accepts the same format of training arguments (e.g. batch size, gradient checkpointing) as before, except that it trains the model for the pairwise reward objective from [the InstructGPT paper](https://arxiv.org/pdf/2203.02155.pdf):

![img](https://i.imgur.com/2JzNAPs.png)

Note that the model itself does not score pairs: it processes chosen ($y_w$) and rejected ($y_l$) samples independently. To minimize this loss, the reward model needs to score chosen sample higher than the rejected one. Note that the formula also assumes some context $x$, which is useful for seq2seq tasks. In our case of movie reviews, $x$ is empty.

In [50]:
import trl

training_args = trl.RewardConfig( # like transformers.TrainingArguments
    output_dir="reward_model",
    per_device_train_batch_size=32,
    gradient_accumulation_steps=1,
    learning_rate=1.41e-5,
    max_steps=1_000,              # note: training may need more than 1k steps
    logging_steps=50,
    gradient_checkpointing=True,  # reduce memory usage but train ~30% slower
    gradient_checkpointing_kwargs={"use_reentrant": False},
    fp16=True                     # disable this on CPU or on very old GPUs
    # you may add any other hyperparameters that you found useful in weeks 5-7
)

trainer = trl.RewardTrainer(
    model=reward_model,
    args=training_args,
    tokenizer=reward_tokenizer,
    train_dataset=reward_data,
    peft_config=None,  # optionally, you may tune with LoRA, prompt-tuning, etc
)

trainer.train()

c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\trl\trainer\reward_trainer.py:182: UserWarning: When using RewardDataCollatorWithPadding, you should set `max_length` in RewardConfig. It will be set to `512` by default, but you should do it yourself in the future.
  warnings.warn(
c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\trl\trainer\reward_trainer.py:199: UserWarning: When using RewardDataCollatorWithPadding, you should set `remove_unused_columns=False` in your RewardConfig we have set it for you, but you should do it yourself in the future.
  warnings.warn(
c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\accelerate\accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs
  0%|          | 0/1000

{'loss': 0.5091, 'grad_norm': 3.0127673149108887, 'learning_rate': 1.34091e-05, 'epoch': 0.0}


 10%|█         | 100/1000 [01:21<10:47,  1.39it/s]

{'loss': 0.1947, 'grad_norm': 2.138087749481201, 'learning_rate': 1.2718200000000001e-05, 'epoch': 0.0}


 15%|█▌        | 150/1000 [01:57<10:10,  1.39it/s]

...

100%|██████████| 1000/1000 [12:07<00:00,  1.40it/s]

{'loss': 0.044, 'grad_norm': 1.3303836584091187, 'learning_rate': 2.82e-08, 'epoch': 0.0}


100%|██████████| 1000/1000 [12:09<00:00,  1.37it/s]

{'train_runtime': 729.3085, 'train_samples_per_second': 43.877, 'train_steps_per_second': 1.371, 'train_loss': 0.10954286599159241, 'epoch': 0.0}


TrainOutput(global_step=1000, training_loss=0.10954286599159241, metrics={'train_runtime': 729.3085, 'train_samples_per_second': 43.877, 'train_steps_per_second': 1.371, 'total_flos': 0.0, 'train_loss': 0.10954286599159241, 'epoch': 0.00020479997902848215})

In [7]:
reward_model.gradient_checkpointing_disable()
reward_model.eval()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

### Sanity-check the reward model (1 point)

Let's check how our reward model performs.

__Your task__ is to measure how often does your reward model can rank a pair of (chosen and rejected) reviews correctly. Please measure this separately for train data (`imdb`) and a separate test set loaded below.

In [8]:

for sample_index in 45, 16000:
  print('TEXT:', imdb[sample_index]['text'])
  inputs = reward_tokenizer(
      imdb[sample_index]['text'], truncation=True, return_tensors='pt').to(device)
  with torch.no_grad():
    reward = reward_model(**inputs).logits[0, 0].item()
    print("REWARD:", reward)
  print('LABEL:', imdb[sample_index]['label'])
  print()

# note: your reward model may produce different absolute rewards.
# This is fine as long as the rewards are ordered correctly (most of the time)

TEXT: This movie sucked. It really was a waste of my life. The acting was atrocious, the plot completely implausible. Long, long story short, these people get "terrorized" by this pathetic "crazed killer", but completely fail to fight back in any manner. And this is after they take a raft on a camping trip, with no gear, and show up at a campsite that is already assembled and completely stocked with food and clothes and the daughters headphones. Additionally, after their boat goes missing, they panic that they're stuck in the woods, but then the daughters boyfriend just shows up and they apparently never consider that they could just hike out of the woods like he did to get to them. Like I said, this movie sucks. A complete joke. Don't let your girlfriend talk you into watching it.
REWARD: 5.3046875
LABEL: 0

TEXT: Good: Engaging cinematic firefights, great presentation, vehicles are actually fun to drive, fairly appealing multiplayer, faithful to the movie, and the list goes on.<br />

In [9]:
from sklearn.metrics import roc_auc_score

In [11]:
from tqdm import tqdm
imdb_test = datasets.load_dataset("imdb", split='test')

rewards = []
labels = []

for idx in tqdm(range(len(imdb_test))):
    inputs = reward_tokenizer(imdb_test[idx]['text'], truncation=True, return_tensors='pt').to(device)
    with torch.no_grad():
        reward = reward_model(**inputs).logits[0, 0].item()
        rewards.append(reward)
        labels.append(imdb_test[idx]['label'])

score = roc_auc_score(labels,rewards)
roc_auc_score(labels,rewards)


# <a whole lot of your code here, feel free to spit it as you see fit>


100%|██████████| 25000/25000 [02:10<00:00, 191.11it/s]


0.0277544896

In [20]:
1-score

0.9722455104

### Reward-guided generation (1 point)

If you did everything right, by now you should have a decent reward model. Before we use it for reinforcement learning, let's see if we can align model samples without any training.

To do so, you can use reward-guided inference: __generate N=16 samples, then select the one with the highest reward__ (according to your reward model).

For this problem, it's on you to demonstrate whether or not your code works. Find at least 5 neutral prompts such as "This movie is" (...), generate samples, rank them based on reward and show which samples get the highest reward.

Note: it is faster to generate samples in parallel, rather than sequentially, as follows:




In [19]:
inputs = main_tokenizer(["It was"] * 5, return_tensors='pt').to(device)
for candidate in main_model.generate(**inputs, max_new_tokens=50, do_sample=True):
  print("Sample:", main_tokenizer.decode(candidate.flatten().cpu().numpy().tolist()))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Sample: It was always my dream to play poker, I always thought I could do it, so I got involved. I think it was interesting to see how popular poker was before getting into poker. Many poker players have been talking about the fact that their poker careers are
Sample: It was funny how quickly he was getting old, so his father gave him out from the hospital and he was okay. The doctor thought he died and he went to the same hospital a couple days later that same day. My daughter did not know anything about this
Sample: It was interesting how she managed to remain credible throughout; her most memorable scenes were the flashbacks where she visits school and falls for an older older child. We have the young girl, who tries to find a place for her in life but her parents turn her into
Sample: It was the first episode where the actors decided to throw in a couple of cheap shots. There is no way there would be another show. There would be worse.<|endoftext|><|endoftext|><|endoftext|><|endoft

In [11]:
reward_model = transformers.AutoModelForSequenceClassification.from_pretrained("reward_model/checkpoint-1000", device_map=device)

In [53]:
# <YOUR CODE HERE> - feel free to organize it as you see fit
outputs = []

inputs = main_tokenizer(["This movie is"] * 16, return_tensors='pt').to(device)

for gen in main_model.generate(**inputs, max_new_tokens=50, do_sample=True):
    outputs.append(main_tokenizer.decode(gen.flatten().cpu().numpy().tolist()))

    

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [54]:
rewards = []
for i in outputs:
    inputs = reward_tokenizer(i, truncation=True, return_tensors='pt').to(device)
    rewards.append(reward_model(**inputs).logits[0,0].detach().cpu())

In [61]:
import numpy as np
np.array(outputs)[np.argsort(rewards)]

array(["This movie is about a girl who loses her virginity accidentally and her sister gets caught up in it. A story I haven't had to think of any more!! A nice twist to this story!! Also, a really nice action/suspense in the last hour with",
       "This movie is definitely worth watching. I'd give it a 9, as you get a nice feel for the characters even if you're not the type to read all about them.<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>",
       'This movie is about two teenagers trying to solve one of the best murder mysteries ever set in Chicago. The leads play "ghosts" from one mystery to the next and the mystery is far removed from what they had done in the first film. A mystery for all',
       'This movie is like a horror movie, as a bad thing, but you do get a really good one that is not even bad, b

# Stage 2: fine-tune the main model with RL


For this tutorial, we will optimize GPT2 to produce positive IMDB movie reviews using the reward model you trained above.

Unlike supervised fine-tuning, RL allows model to generate it's own sentences on each training step. Then, it calculates the reward of those specific sentences, and finally, updates the model to increase the probability of sentences with high reward.

Thus, each RLHF consists of three stages: __Rollout__, __Evaluation__ and __Update__

<div style="text-align: center">
<img src='https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/images/gpt2_bert_training.png' width='600'>

The update stage depends on the specific RL algorithm. We'll be using Proximal Policy Optimization, or [PPO](https://arxiv.org/abs/1707.06347), similarly to what was used for InstructGPT.

Before we run those 3 stages, however, we need to create a dataset of "queries" - partial reviews in our case.

In [6]:
import trl
# Note: this code is specific to IMDB; you will need to re-write it for other tasks
imdb_for_rlhf = imdb.filter(lambda row: len(row['text']) > 200, batched=False)
imdb_for_rlhf = imdb_for_rlhf.remove_columns(['label'])
sample_length = trl.core.LengthSampler(2, 8)  # use the first 2-8 tokens as query

def select_query_and_tokenize(sample):
    query_ids = main_tokenizer.encode(sample["text"])[: sample_length()]
    sample["query"] = main_tokenizer.decode(query_ids)  # query is the only required column
    sample["input_ids"] = query_ids  # to avoid re-tokenizing later
    return sample  # we do not need the rest - it will be generated by the model

imdb_for_rlhf = imdb_for_rlhf.map(select_query_and_tokenize, batched=False)
imdb_for_rlhf.set_format(type="torch")

Next, let's prepare your reward model to predict rewards on whatever reviews were generated. Note that we use plaintext reviews because main model uses a different tokenizer from the reward model.

In [7]:
from typing import List
def compute_reward(texts: List[str]) -> torch.Tensor:
  inputs = reward_tokenizer(texts, truncation=True, padding=True, return_tensors='pt').to(device)
  with torch.no_grad():
    return reward_model(**inputs).logits[:, 0]

In [12]:
compute_reward([imdb[45]['text'], imdb[16000]['text']])  # test on human-written reviews

tensor([ 5.3037, -4.7717], device='cuda:0')

Finally, we move to RL training. In this tutorial, we'll train LoRA adapters and not the full model.

In [13]:
import peft
peft_config = peft.LoraConfig(
    task_type=peft.TaskType.CAUSAL_LM, r=32, lora_alpha=32, lora_dropout=0.0, inference_mode=False
)

# reload main model as AutoModelForCausalLMWithValueHead - with an extra head needed for PPO
main_tokenizer = transformers.AutoTokenizer.from_pretrained("lvwerra/gpt2-imdb")
main_tokenizer.pad_token = main_tokenizer.eos_token

main_model = trl.AutoModelForCausalLMWithValueHead.from_pretrained("lvwerra/gpt2-imdb", device_map=device)
main_model = peft.get_peft_model(main_model, peft_config, adapter_name='default')
main_model.print_trainable_parameters()

trainable params: 1,179,648 || all params: 125,620,225 || trainable%: 0.9391


c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\peft\tuners\lora\layer.py:1091: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


Same as before, trl has a special type of trainer that minimize PPO-specific pseudo-loss. You can read more on this trainer [here](https://huggingface.co/docs/trl/main/en/ppo_trainer).

In [14]:
training_args = trl.PPOConfig(
    model_name=main_model.config._name_or_path,
    gradient_accumulation_steps=4,
    learning_rate=1.41e-5,
    batch_size=64,
    mini_batch_size=16,
    ppo_epochs=4,                 # PPO performs this many updates per training batch
)

ppo_trainer = trl.PPOTrainer(
    training_args, model=main_model.model, tokenizer=main_tokenizer,
    dataset=imdb_for_rlhf, data_collator=lambda data: dict((key, [d[key] for d in data]) for key in data[0])
)  # note: we pass main_model.model because PPOTrainer checks for one of several supported model types ...
# ... main_model.model is a model with adapters, which is supported. main_model itself is a wrapper that is not supported

c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\trl\trainer\ppo_config.py:207: FutureWarning: `PPOConfig` is deprecated and will be removed in the future. Please use `PPOv2Config` with `PPOv2Trainer` instead.
  warnings.warn(
c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\trl\trainer\ppo_trainer.py:193: FutureWarning: `PPOTrainer` is deprecated and will be removed in trl v0.12. Please us

In [15]:
from tqdm.auto import tqdm
max_steps = 50   # can be insufficient for some tasks - watch your learning curves
generation_kwargs = dict(
    min_length=-1, max_new_tokens=128, do_sample=True, top_k=0, top_p=1.0, pad_token_id=main_tokenizer.eos_token_id)
#                                  ^-- task-specific parameter!
with tqdm(enumerate(ppo_trainer.dataloader), total=max_steps) as progressbar:
  # note: ppo_trainer.dataloader is just a regular dataloader of queries, no RL-specific magic :)
  for epoch, batch in progressbar:
    if epoch >= max_steps:
        break

    # Rollout stage: generate continuations from batch queries using main_model
    response_tensors = ppo_trainer.generate(batch['input_ids'], **generation_kwargs)
    # ^-- list of tensors of token ids from main model tokenizer

    # de-tokenize responses to strings (since reward model uses a different tokenizer)
    batch["response"] = [main_tokenizer.decode(response.squeeze()) for response in response_tensors]
    # note: response_tensors already contain query tokens, so we don't need to add queries manually.
    # This may not be true for other tasks: check this manually by viewing batch["response"] and batch["query"]


    # Evaluation stage
    rewards = compute_reward(batch['response'])

    # Update stage
    stats = ppo_trainer.step(batch['input_ids'], response_tensors, list(rewards.split(1)))
    stats['rewards/mean'] = rewards.mean().item()

    print("-" * 30, 'STEP', epoch, '-' * 30)
    print(f'rewards/mean:\t{stats["rewards/mean"]:.9f}\t<---- average reward over this batch (higher=better, noisy)')
    print(f'ppo/returns/mean:\t{stats["ppo/returns/mean"]:.9f}\t<---- model-estimated average discounted reward')
    print(f'objective/kl:\t{stats["objective/kl"]:.9f}\t<---- how far we are from the original model (regularizer)')
    print()

    ppo_trainer.log_stats(stats, batch, list(rewards.split(1)))

  2%|▏         | 1/50 [00:22<18:12, 22.30s/it]

------------------------------ STEP 0 ------------------------------
rewards/mean:	0.537303925	<---- average reward over this batch (higher=better, noisy)
ppo/returns/mean:	0.027607590	<---- model-estimated average discounted reward
objective/kl:	0.000000000	<---- how far we are from the original model (regularizer)



  4%|▍         | 2/50 [00:43<17:33, 21.95s/it]

------------------------------ STEP 1 ------------------------------
rewards/mean:	-0.114208698	<---- average reward over this batch (higher=better, noisy)
ppo/returns/mean:	-0.073779672	<---- model-estimated average discounted reward
objective/kl:	0.000067968	<---- how far we are from the original model (regularizer)



  6%|▌         | 3/50 [01:05<16:57, 21.65s/it]

...

------------------------------ STEP 47 ------------------------------
rewards/mean:	0.925737619	<---- average reward over this batch (higher=better, noisy)
ppo/returns/mean:	0.166211829	<---- model-estimated average discounted reward
objective/kl:	0.396295190	<---- how far we are from the original model (regularizer)



 98%|█████████▊| 49/50 [17:48<00:21, 21.54s/it]

------------------------------ STEP 48 ------------------------------
rewards/mean:	1.073199272	<---- average reward over this batch (higher=better, noisy)
ppo/returns/mean:	0.179797187	<---- model-estimated average discounted reward
objective/kl:	0.731850266	<---- how far we are from the original model (regularizer)



100%|██████████| 50/50 [18:12<00:00, 21.85s/it]

------------------------------ STEP 49 ------------------------------
rewards/mean:	0.493699908	<---- average reward over this batch (higher=better, noisy)
ppo/returns/mean:	0.101782501	<---- model-estimated average discounted reward
objective/kl:	0.935232162	<---- how far we are from the original model (regularizer)



## Main assignment - <u>actually</u> train the model (8 points)


Your main task for this week is to use the RLHF pipeline to train a model for a reward of your choice. Here's what you can choose from:

__A. Toxicity fine-tuning:__ train the model to be less (or more!) toxic. For this task, you may use the data from [jigsaw toxic comments](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge) and [lmsys/toxic-chat](https://huggingface.co/datasets/lmsys/toxic-chat),  or any other source. Alternatively, you may use toxicity scores from [oasst1](https://huggingface.co/datasets/OpenAssistant/oasst1).


__B. Actual human feedback:__ use one of the existing datasets with pairwise human feedback to align your langauge model. You may use [anthropic's hh-rlhf](https://huggingface.co/datasets/Anthropic/hh-rlhf), [OpenAssistant dataset](https://huggingface.co/datasets/OpenAssistant/oasst1) or any other data you see fit. You may also turn the tables and train the model to [minimize](https://habrastorage.org/getpro/geektimes/post_images/ac7/2ad/827/ac72ad82767d4132164a4b6b76196c42.jpg) human preferences, as long as your model does not degrade to gibberish.

__C. Controlled generation:__ Instead of training a reward model from human feedback, you may define the reward function as the text length (longer or shorter) or number of times the model uses specific words (e.g. "sorry", "apologize"). If you choose specific words, make sure the model generates them at least sometimes.

__Alternatively,__ you may choose a different task. However, unless your task is very similar to one of the above, there is a chance that it will be **significantly** harder to solve, requiring orders of magnitude more compute and tuning. If you are in doubt, please ask the course staff. If they are AFK (again >.<), please prefer one of the recommended tasks.


#### General tips & tricks


Things to look out for:
- during PPO stage, the reward model should be in eval mode (dropout disabled)
- make sure max_length and max_new_tokens are enough for your chosen dataset - at least most of the time
- when in doubt, view the data manually or inspect how the model performs on a few samples


We highly recommend that you manually check the performance after each sub-stage:
1. when you assembled the pairwise dataset, inspect a couple of from of *your* dataset class and detokenize them. Make sure that you-the-human understand why one sample was accepted and the other - rejected. At least most of the time. This also lets you spot tokenization/truncation errors.
2. after you trained a reward model, measure how accurate this model is in isolation. If your reward model is poor, any subsequent RLHF will also fail.
3. once you've trained the main model with RL, ask it to generate examples and explore how well it does. If it produces an obviously bad output, check if the reward model assigns high reward to that output. If yes, reward model is the culprit; if no, it's a question of better/longer PPO training.

__It is also a good idea to periodically print samples during training.__

__When stuck, simplify the problem.__ If you've spent a several hours enchanting the reward model but it still won't budge, try switching to a simple subtask. For instance, if you're training on hh-rlhf, try limiting it the dataset to 10% of the shortest sequences - they are typically easier to learn.


## Assignment stages (and grading)

Regardless of the specific task you chose, your solution needs to contain several parts that will be graded separately.


#### Stage 1: reward model (4 points)

Construct a dataset for training the reward model on your problem. Then, train a reward model on that dataset and evaluate how well can your model predict preferences on a hold-out (test) subset of your data.

Please make sure that the part of your notebook where you evaluate reward model is clearly visible and reasonably easy to read. And for all that is holy, do not call it IMDB unless it actually **is** data of imdb movie reviews :)

__Not all tasks require a reward model for later PPO fine-tuning.__ For instance, there's no reason to train a reward model if your reward equals sentence length. Likewise, toxicity reward can be estimated with a pre-trained toxicity classifier. __If your task does not require training a reward model, please train an unrelated model on [hh-rlhf](https://huggingface.co/datasets/Anthropic/hh-rlhf) as though you were solving assignment version B.__ This is for grading purposes only, you won't use this model for stage 2.


#### Stage 2: RL fine-tuning (4 points)

Once the reward model is ready - or you can compute rewards without a model - it is time to maximize that reward with PPO. Optionally, you may replace PPO with another RL algorithm (or unlikelihood learning scheme), but only if you're feeling adventurous.


First, you need to choose a language model to be fine-tuned. You may choose any model, but make sure that your model **can** generate the data in your format. For instance, [Mistral-7B](https://huggingface.co/mistralai/Mistral-7B-v0.1) is a general purpose LM and may (or may not) need prompt engineering to generate chat assistant responses. For that reason, it is best if you **do not use `"lvwerra/gpt2-imdb"` unless you're generating only movie reviews**.



There are two "difficulty modes" for this task:
For the **easy mode**, use [gpt2-large](https://huggingface.co/gpt2-large) or [opt-1.3b](https://huggingface.co/facebook/opt-1.3b) with minimal code changes.
If you want the **Hard mode:** use a larger (e.g. 7B) model in combination with `load_in_4bit` and LoRA, the same way we did last week.
Some reasonable model choices are [LLaMA-7B](https://huggingface.co/Enoch/llama-7b-hf), [Falcon-7b](https://huggingface.co/tiiuae/falcon-7b), [Mistral-7B](https://huggingface.co/mistralai/Mistral-7B-v0.1) for general-purpose LM or [guanaco-7b](https://huggingface.co/timdettmers/guanaco-7b), [vicuna-7b](https://huggingface.co/lmsys/vicuna-7b-v1.5) for chat-based tasks, though there are many more (see [leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard)). In the hard mode, you will need to modify the training arguments to enable 4-bit fine-tuning. Furthermore, your experiments will take somewhat longer to complete. On the plus side, your model will produce significantly better results.

__High reward is not enough!__ RL algorithms are famous for [cheating their reward functions](https://openai.com/research/faulty-reward-functions). To ensure that your model is actually doing what you want it to do, you will need some additional evaluation. To get the full grade, provide at least 20 side-by-side examples of your fine-tuned model vs original model predictions and a short summary.

Alternatively, you may provide 5 examples and some extrinsic evaluation metric over many examples. For instance, you may use a different pre-trained toxicity score for option A. When dealing with human preferences, you may choose to [enlist actual humans](https://toloka.ai/) or [ask GPT4/Claude](https://arxiv.org/pdf/2304.03277.pdf) to compare your model's predictions. For task C, when optimizing for simple rewards like sentence lengths, it is enough to compare histograms of rewards (e.g. average lengths).












In [43]:
from datasets import load_dataset

ds = load_dataset("lmsys/toxic-chat", "toxicchat0124")

In [44]:
import torch
import numpy as np
import datasets

class FeaturedDataset(torch.utils.data.Dataset):
    """A dataset with pairwise ranked numerical features"""
    def __init__(self, ds, tokenizer, target_feature: int):
        super().__init__()
        self.tokenizer = tokenizer
        self.ds = ds
        self.target_feature = target_feature

    def __len__(self):
        return len(self.ds)**2  # all pairs

    def __getitem__(self, index: int):
        sample = self.ds[index//len(self.ds)]
        another = self.ds[index%len(self.ds)]
        if dict(eval(sample['openai_moderation']))[self.target_feature] > dict(eval(another['openai_moderation']))[self.target_feature]:
            chosen = self.tokenizer(sample['model_output'], truncation=True)
            rejected = self.tokenizer(another['model_output'], truncation=True)
        else:
            chosen = self.tokenizer(another['model_output'], truncation=True)
            rejected = self.tokenizer(sample['model_output'], truncation=True)
            
        return dict(input_ids_chosen=chosen['input_ids'], attention_mask_chosen=chosen['attention_mask'],
                    input_ids_rejected=rejected['input_ids'], attention_mask_rejected=rejected['attention_mask'])

In [45]:

import transformers
device = 'cuda'
reward_model = transformers.AutoModelForSequenceClassification.from_pretrained("distilbert-base-cased", device_map=device)
reward_tokenizer = transformers.AutoTokenizer.from_pretrained("distilbert-base-cased")
reward_data = FeaturedDataset(ds['train'],reward_tokenizer,'sexual')

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
import trl

training_args = trl.RewardConfig( # like transformers.TrainingArguments
    output_dir="r_model",
    per_device_train_batch_size=100,
    gradient_accumulation_steps=2,
    learning_rate=2e-5,
    max_steps=2000,            # note: training may need more than 1k steps
    logging_steps=50,
    gradient_checkpointing=True,  # reduce memory usage but train ~30% slower
    gradient_checkpointing_kwargs={"use_reentrant": False},
    fp16=True,
    dataset_num_proc=11,
    
)

trainer = trl.RewardTrainer(
    model=reward_model,
    args=training_args,
    tokenizer=reward_tokenizer,
    train_dataset=reward_data,
    peft_config=None,  # optionally, you may tune with LoRA, prompt-tuning, etc
)

trainer.train()

c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\trl\trainer\reward_trainer.py:182: UserWarning: When using RewardDataCollatorWithPadding, you should set `max_length` in RewardConfig. It will be set to `512` by default, but you should do it yourself in the future.
  warnings.warn(
c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\trl\trainer\reward_trainer.py:199: UserWarning: When using Rew

{'loss': 0.6349, 'grad_norm': 1.9936622381210327, 'learning_rate': 1.95e-05, 'epoch': 0.0}


  5%|▌         | 100/2000 [07:57<2:28:29,  4.69s/it]

{'loss': 0.553, 'grad_norm': 2.1092488765716553, 'learning_rate': 1.9010000000000003e-05, 'epoch': 0.0}


  8%|▊         | 150/2000 [11:50<2:17:55,  4.47s/it]

...

100%|██████████| 2000/2000 [2:37:41<00:00,  4.35s/it]

{'loss': 0.1377, 'grad_norm': 3.4875423908233643, 'learning_rate': 1e-08, 'epoch': 0.02}


100%|██████████| 2000/2000 [2:37:42<00:00,  4.73s/it]

{'train_runtime': 9462.5938, 'train_samples_per_second': 42.272, 'train_steps_per_second': 0.211, 'train_loss': 0.229576851606369, 'epoch': 0.02}


TrainOutput(global_step=2000, training_loss=0.229576851606369, metrics={'train_runtime': 9462.5938, 'train_samples_per_second': 42.272, 'train_steps_per_second': 0.211, 'total_flos': 0.0, 'train_loss': 0.229576851606369, 'epoch': 0.015487787879257206})

In [1]:
from datasets import load_dataset

ds = load_dataset("lmsys/toxic-chat", "toxicchat0124")

c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [46]:
import transformers
device = 'cuda'
reward_tokenizer = transformers.AutoTokenizer.from_pretrained("distilbert-base-cased")
reward_model = transformers.AutoModelForSequenceClassification.from_pretrained("r_model/checkpoint-2000", device_map=device)

In [3]:
import trl
import torch
import transformers
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
qconfig = transformers.BitsAndBytesConfig(load_in_4bit=True,bnb_4bit_use_double_quant=True)
main_tokenizer = transformers.AutoTokenizer.from_pretrained("openai-community/gpt2-large")
main_tokenizer.pad_token = main_tokenizer.eos_token
main_model = trl.AutoModelForCausalLMWithValueHead.from_pretrained("openai-community/gpt2-large", device_map=device,quantization_config = qconfig,torch_dtype = torch.float32)


In [4]:
from peft import get_peft_model,LoraConfig,TaskType
lora_config = LoraConfig(
                target_modules=["c_attn",'c_proj','c_fc','lm_head'],
                lora_alpha=32,
                inference_mode=False,
                use_rslora = True,
                bias = 'all',
                lora_dropout=0.,
                r=4,
                task_type=TaskType.CAUSAL_LM,
                init_lora_weights='gaussian'
            )
main_model = get_peft_model(main_model,lora_config)

In [5]:
main_model.print_trainable_parameters()

trainable params: 3,663,429 || all params: 777,186,629 || trainable%: 0.4714


In [6]:
import trl
# Note: this code is specific to IMDB; you will need to re-write it for other tasks
sample_length = trl.core.LengthSampler(2, 8)  # use the first 2-8 tokens as query

def select_query_and_tokenize(sample):
    query_ids = main_tokenizer.encode(sample["user_input"])[:sample_length()]
    sample["query"] = main_tokenizer.decode(query_ids)  # query is the only required column
    sample["input_ids"] = query_ids  # to avoid re-tokenizing later
    return sample  # we do not need the rest - it will be generated by the model

ds_rlhf = ds['test'].map(select_query_and_tokenize, batched=False)
ds_rlhf.set_format(type="torch")

In [7]:
import trl
training_args = trl.PPOConfig(
    model_name=main_model.config._name_or_path,
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    batch_size=32,
    mini_batch_size=8,
    ppo_epochs=4,
    dataset_num_proc=8,
              # PPO performs this many updates per training batch
)

ppo_trainer = trl.PPOTrainer(
    training_args, model=main_model.model, tokenizer=main_tokenizer,
    dataset=ds_rlhf, data_collator=lambda data: dict((key, [d[key] for d in data]) for key in data[0]),
)  # note: we pass main_model.model because PPOTrainer checks for one of several supported model types ...
# ... main_model.model is a model with adapters, which is supported. main_model itself is a wrapper that is not supported

c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\trl\trainer\ppo_config.py:207: FutureWarning: `PPOConfig` is deprecated and will be removed in the future. Please use `PPOv2Config` with `PPOv2Trainer` instead.
  warnings.warn(
c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\trl\trainer\ppo_trainer.py:193: FutureWarning: `PPOTrainer` is deprecated and will be removed in trl v0.12. Please us

In [8]:
from typing import List
def compute_reward(texts: List[str]) -> torch.Tensor:
  inputs = reward_tokenizer(texts, truncation=True, padding=True, return_tensors='pt').to(device)
  with torch.no_grad():
    return reward_model(**inputs).logits[:, 0]

In [9]:
from tqdm.auto import tqdm
max_steps = 150   # can be insufficient for some tasks - watch your learning curves
generation_kwargs = dict(
    min_length=-1, max_new_tokens=128, do_sample=True, top_k=0, top_p=1.0, pad_token_id=main_tokenizer.eos_token_id)
#                                  ^-- task-specific parameter!
with tqdm(enumerate(ppo_trainer.dataloader), total=max_steps) as progressbar:
  # note: ppo_trainer.dataloader is just a regular dataloader of queries, no RL-specific magic :)
  for epoch, batch in progressbar:
    if epoch >= max_steps:
        break
    # Rollout stage: generate continuations from batch queries using main_model
    response_tensors = ppo_trainer.generate(batch['input_ids'], **generation_kwargs)
    # ^-- list of tensors of token ids from main model tokenizer
    # de-tokenize responses to strings (since reward model uses a different tokenizer)
    batch["response"] = [main_tokenizer.decode(response.squeeze()) for response in response_tensors]
    # note: response_tensors already contain query tokens, so we don't need to add queries manually.
    # This may not be true for other tasks: check this manually by viewing batch["response"] and batch["query"]
    # Evaluation stage
    rewards = compute_reward(batch['response'])
    # Update stage
    stats = ppo_trainer.step(batch['input_ids'], response_tensors, list(rewards.split(1)))
    stats['rewards/mean'] = rewards.mean().item()

    print("-" * 30, 'STEP', epoch, '-' * 30)
    print(f'rewards/mean:\t{stats["rewards/mean"]:.9f}\t<---- average reward over this batch (higher=better, noisy)')
    print(f'ppo/returns/mean:\t{stats["ppo/returns/mean"]:.9f}\t<---- model-estimated average discounted reward')
    print(f'objective/kl:\t{stats["objective/kl"]:.9f}\t<---- how far we are from the original model (regularizer)')
    print()
    if epoch % 10 == 0:
      for name, param in main_model.named_parameters():
        if param.requires_grad:
            torch.save(param,f"ft/{name}")
    ppo_trainer.log_stats(stats, batch, list(rewards.split(1)))
  for name, param in main_model.named_parameters():
      if param.requires_grad:
          torch.save(param,f"ft/{name}")

    

  1%|          | 1/150 [01:03<2:37:43, 63.51s/it]

------------------------------ STEP 0 ------------------------------
rewards/mean:	1.341312528	<---- average reward over this batch (higher=better, noisy)
ppo/returns/mean:	0.226613402	<---- model-estimated average discounted reward
objective/kl:	0.000000000	<---- how far we are from the original model (regularizer)



  1%|▏         | 2/150 [02:15<2:49:08, 68.57s/it]

------------------------------ STEP 1 ------------------------------
rewards/mean:	1.314770699	<---- average reward over this batch (higher=better, noisy)
ppo/returns/mean:	0.319299579	<---- model-estimated average discounted reward
objective/kl:	-0.634562492	<---- how far we are from the original model (regularizer)



c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\trl\trainer\ppo_trainer.py:1313: UserWarning: KL divergence is starting to become negative: -1.27 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(
  2%|▏         | 3/150 [03:24<2:47:50, 68.51s/it]

...

------------------------------ STEP 147 ------------------------------
rewards/mean:	8.776826859	<---- average reward over this batch (higher=better, noisy)
ppo/returns/mean:	5.799866676	<---- model-estimated average discounted reward
objective/kl:	20.845817566	<---- how far we are from the original model (regularizer)



 99%|█████████▉| 149/150 [1:20:04<00:11, 11.42s/it]

------------------------------ STEP 148 ------------------------------
rewards/mean:	9.053957939	<---- average reward over this batch (higher=better, noisy)
ppo/returns/mean:	6.131712914	<---- model-estimated average discounted reward
objective/kl:	19.449970245	<---- how far we are from the original model (regularizer)



100%|██████████| 150/150 [1:20:17<00:00, 32.12s/it]

------------------------------ STEP 149 ------------------------------
rewards/mean:	8.810099602	<---- average reward over this batch (higher=better, noisy)
ppo/returns/mean:	5.674315453	<---- model-estimated average discounted reward
objective/kl:	20.663236618	<---- how far we are from the original model (regularizer)



In [10]:
import trl
import torch
import transformers
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
qconfig = transformers.BitsAndBytesConfig(load_in_4bit=True,bnb_4bit_use_double_quant=True)
main_tokenizer = transformers.AutoTokenizer.from_pretrained("openai-community/gpt2-large")
main_tokenizer.pad_token = main_tokenizer.eos_token
main_model = trl.AutoModelForCausalLMWithValueHead.from_pretrained("openai-community/gpt2-large", device_map=device,quantization_config = qconfig,torch_dtype = torch.float32)

from peft import get_peft_model,LoraConfig,TaskType
lora_config = LoraConfig(
                target_modules=["c_attn",'c_proj','c_fc','lm_head'],
                lora_alpha=32,
                inference_mode=False,
                use_rslora = True,
                bias = 'all',
                lora_dropout=0.,
                r=4,
                task_type=TaskType.CAUSAL_LM,
                init_lora_weights='gaussian'
            )
main_model = get_peft_model(main_model,lora_config)
main_model.print_trainable_parameters()

trainable params: 3,663,429 || all params: 777,186,629 || trainable%: 0.4714


In [11]:
for name, param in main_model.named_parameters():
    if param.requires_grad:
        param.requires_grad_(False)
        param.copy_(torch.load(f"ft/{name}"))

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_16416\1397046154.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  param.copy_(torch.load(f"ft/{name}"))


In [97]:
import warnings
warnings.filterwarnings('ignore')

In [102]:
gens = main_model.model.pretrained_model.generate(**main_tokenizer(["you know"]*15,return_tensors='pt'), max_new_tokens=50, do_sample=True)

for i in range(15):
    eos_idx = gens[i].tolist().index(main_tokenizer.pad_token_id)
    gen = main_tokenizer.decode(gens[i,:eos_idx].flatten().detach().cpu().tolist())
    reward = reward_model(reward_tokenizer(gen,return_tensors='pt')['input_ids'].cuda())['logits'][0,0].item()
    print(gen,reward)




Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


you know of the way you want to masturbate. 9.679192543029785
you know I love kissing girls. 9.867589950561523
you know her penis. 9.499441146850586
you know how sex is. It's an orgasm. 9.90887451171875
you know where it was that got sex. 9.739014625549316
you know what the woman said. This wasn't sex." 9.586249351501465
you know, with sexual intercourse, sex, that is it. 9.430999755859375
you know that you have sex. 9.84558391571045
you know what they're used for. They're good quality sex. 9.89096736907959
you know what will make the girl have sex. 9.96318244934082
you know him as the adult male." 9.542046546936035
you know how she wants her panties to feel. 9.84951400756836
you know how sexually sexy you look. 9.829161643981934
you know the reason I want to fuck her. 9.712608337402344
you know. Do something. Let them fuck, now. 8.319439888000488
